import os
#os.environ['create_image']='True'# Train mobilenet image classifier
Currenly uses mobilenet V2 to train an image classifier.

Future work:  
[] support different deep learning neural network architectures (like VGG, ResNet, ...)  
[] expose hyper parameters of those architectures as input to this component  
[] support hyperparameter tuning (iteally using Katib)  

In [1]:
import os
import sys
#os.environ['create_image']='True'
os.environ['repository']='romeokienzler'
os.environ['version']='0.1'
#
#os.environ['install_requirements']='True'

In [3]:
import os
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM registry.access.redhat.com/ubi8/python-38
    RUN pip install nvflare==2.0.16
    RUN pip install tensorflow==2.9.1
    ADD nvflare.ipynb /
    #ENTRYPOINT ["ipython","/nvflare.ipynb"]
    """
    with open("Dockerfile", "w") as text_file: 
        text_file.write(docker_file)

    !docker build -t claimed-train-nvflare:`echo $version` .
    !docker tag claimed-train-nvflare:`echo $version` `echo $repository`/claimed-train-nvflare:`echo $version`
    !docker push `echo $repository`/claimed-train-nvflare:`echo $version`
elif bool(os.environ.get('install_requirements',False)):
    !pip install nvflare==2.0.16 tensorflow==2.9.1

Sending build context to Docker daemon  72.19kB
Step 1/4 : FROM registry.access.redhat.com/ubi8/python-38
 ---> fd4f06020ce7
Step 2/4 : RUN pip install nvflare==2.0.16
 ---> Using cache
 ---> 1b4a2b31335b
Step 3/4 : RUN pip install tensorflow==2.9.1
 ---> Using cache
 ---> 0562edb5a8ed
Step 4/4 : ADD nvflare.ipynb /
 ---> Using cache
 ---> d5eddadd6193
Successfully built d5eddadd6193
Successfully tagged claimed-train-nvflare:0.1
The push refers to repository [docker.io/romeokienzler/claimed-train-nvflare]

274dd023: Preparing 
da8b6e89: Preparing 
2b13e91b: Preparing 
71335733: Preparing 
cbc5f0d9: Preparing 
3ed1330a: Waiting g 
11fd02f0: Preparing 
274dd023: Pushed lready exists 1kB0.1: digest: sha256:a5017c4a58565bfe45e49810bdeac45d72f61b78c3c2e34e5c82a5cea3506f86 size: 2007


In [ ]:
!docker tag claimed-train-nvflare echo $repository`/claimed-train-nvflare


In [ ]:
import os
import tensorflow as tf
# from tensorflow.keras.applications import ResNet50V2 # , MobileNetV3Small
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import Model
from claimed_utils import unzip, zipdir
import os.path
import glob
from io import BytesIO
from minio import Minio
from ansible.module_utils.parsing.convert_bool import boolean

In [ ]:
# @dependency codait_utils.ipynb
# @param image_shape shape the images shall be scaled to and the
# model then will accept
# @param model zip file name
# @param data zip file name
# @param model folder name
# @param data folder name
# @param epochs number of epochs to train
# @param checkpoint activate checkpointing
# @param checkpoint_ip minio endpoint
# @param checkpoint_user minio user
# @param checkpoint_pass minio pw
# @param checkpoint_bucket minio bucket
# @returns model zip file

In [ ]:
image_shape = os.environ.get('image_shape', '400,400')
model_zip = os.environ.get('model_zip', 'model.zip')
data_zip = os.environ.get('data_zip', 'data.zip')
model_folder = os.environ.get('model', 'model')
data = os.environ.get('data', 'data')
epochs = int(os.environ.get('epochs', 1))
checkpoint = boolean(os.environ.get('checkpoint', False))
checkpoint_ip = os.environ.get('checkpoint_ip')
checkpoint_user = os.environ.get('checkpoint_user', 'minio')
checkpoint_pass = os.environ.get('checkpoint_pass', 'minio123')
checkpoint_bucket = os.environ.get('checkpoint_bucket', 'checkpoint')

In [ ]:
exists = False

if checkpoint:
    client = Minio(checkpoint_ip,
                   checkpoint_user,
                   checkpoint_pass,
                   secure=False)

    objects = client.list_objects(checkpoint_bucket)
    asset_name = model_zip
    for obj in objects:
        if asset_name == obj.object_name:
            exists = True
            client.fget_object(checkpoint_bucket, model_zip, model_zip)
            break

In [ ]:
if not exists:
    unzip('.', data_zip)

In [ ]:
if not exists:
    folder = glob.glob(data + "/*")
    num_classes = len(folder)

In [ ]:
if not exists:
    batch_size = 32
    input_shape = 'dummy'  # make the compiler happy
    exec('input_shape = (' + image_shape + ')')

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        'data',
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=input_shape,
        batch_size=batch_size)

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        'data',
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=input_shape,
        batch_size=batch_size)

    train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))
    val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))

In [ ]:
def my_net(model, num_classes, freeze_layers=10, full_freeze='N'):
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes, activation='softmax')(x)
    model_final = Model(model.input, out)
    if full_freeze != 'N':
        for layer in model.layers[0:freeze_layers]:
            layer.trainable = False
    return model_final

In [ ]:
# model = ResNet50V2(weights='imagenet',include_top=False)
# model = my_net(model)

In [ ]:
if not exists:
    exec('input_shape = (' + image_shape + ',3)')

    model = tf.keras.applications.MobileNetV2(
        input_shape=input_shape, alpha=1.0, include_top=False,
        input_tensor=None, pooling=None, classes=num_classes,
        classifier_activation='softmax'
    )
    model = my_net(model, num_classes=num_classes)

In [ ]:
# model = tf.keras.applications.VGG16(
#     include_top=True, weights=None, input_tensor=None,
#     input_shape=(244, 244, 3), pooling=None, classes=2,
#     classifier_activation='softmax'
# )
# model = my_net(model)

In [ ]:
if not exists:
    model.compile(
        optimizer="adam",
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

In [ ]:
if not exists:
    model.fit(
        train_ds,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=val_ds
    )
    model.save(model_folder)
    zipdir(model_zip, model_folder)
else:
    print('Model cached, skipping training')

In [ ]:
if not exists:
    size = os.path.getsize(model_zip)
    with open(model_zip, 'rb') as fh:
        buf = BytesIO(fh.read())
        result = client.put_object(
            checkpoint_bucket, model_zip, buf, length=size
        )